In [15]:
from keystone import *
from unicorn import *
from unicorn.x86_const import *
import pandas as pd
from tqdm import tqdm
from IPython.core.debugger import set_trace
from capstone import *
import struct

md = Cs(CS_ARCH_X86, CS_MODE_32)
ks = Ks(KS_ARCH_X86, KS_MODE_32)

In [16]:
# memory address where emulation starts
ADDRESS = 0x0

# plaintext samples
plaintexts=[]

In [22]:
source = """
    # original google asm source
    jmp .forward
    .back:

    pop edi                  # edi has function ptr to .forward -> cipher function
    push byte 4              # push 4 to the stack
    pop eax                  # pop 4 off the stack to eax
    cdq                      # convert double to quad -> sign extension of value in eax to edx: The CDQ instruction copies the sign (bit 31) of the value in the EAX register into every bit position in the EDX register.
    mov ebx, edx             # edx = 0 -> ebx = 0
    inc ebx                  # ebx = 1
    
    mov ebp, 0x474f4f47     # initialize ebp -> 'GOOG'
    #mov ebp, 0x41524541      # initialize ebp -> 'AREA'
    
    push 0xe06cafbd          # push dw to the stack and decrypt it in-place on the stack
    call edi
    push 0xde8d7d56
    call edi
    push 0x47874e62
    call edi
    push 0x65974452
    call edi
    push 0x7e8d424b
    call edi
    push 0x3e59ec52
    call edi
    push 0xab38aea6
    call edi
    push 0x504d26ed
    call edi
    push 0x1e674b49
    call edi
    push 0x254da584
    call edi
    push 0x5449681c
    call edi
    push 0xa19b108e
    call edi
    push 0xada96c07
    call edi

    mov ecx, esp
    int 0x80
    mov eax, ebx
    int 0x80

    .forward: call .back
    xor [esp+4], esi
    add edx, 4
    xor ebp, esi
    ror ebp, 3
    sub ebp, edx
    xor esi, ebp
    ret
    """

In [23]:
def assemble(source):
    try:
        encoding, count = ks.asm(source, 0)
    except KsError as e:
        print(f"failed during assembling.. reason:{str(e)}")

    return bytes(encoding)

In [24]:
# callback for tracing instructions
def hook_code(uc, address, size, user_data):

    # read this instruction code from memory
    tmp = uc.mem_read(address, size)

    for i in md.disasm(tmp, 0):
        
        if i.mnemonic == 'xor' and 'esi' in i.op_str and 'dword ptr' in i.op_str:
            
            esi = uc.reg_read(UC_X86_REG_ESI)
            xor_keys.append(esi)
            esp = uc.reg_read(UC_X86_REG_ESP)
            val = uc.mem_read(esp+4,4)
    
            #import pdb; pdb.set_trace()
            print(f"tracing {i.mnemonic} {i.op_str}: esp + 4: {val.hex()}, esi: {hex(esi)}")
            
            
        if i.mnemonic == 'add' and 'edx' in i.op_str and '4' in i.op_str:     
            esp = uc.reg_read(UC_X86_REG_ESP)
            val = uc.mem_read(esp+4,4)
            print(f"esp + 4 after XOR: {val.hex()}, {str(val)}\n")
            
# callback for tracing Linux interrupt
def hook_intr(uc, intno, user_data):
    # only handle Linux syscall
    if intno != 0x80:
        print("got interrupt %x ???" %intno);
        uc.emu_stop()
        return

    eax = uc.reg_read(UC_X86_REG_EAX)
    eip = uc.reg_read(UC_X86_REG_EIP)

    if eax == 1:    # sys_exit
        #print(">>> 0x%x: interrupt 0x%x, EAX = 0x%x" %(eip, intno, eax))
        uc.emu_stop()
    elif eax == 4:    # sys_write
        
        # ECX = buffer address
        ecx = uc.reg_read(UC_X86_REG_ECX)
        
        # EDX = buffer size
        edx = uc.reg_read(UC_X86_REG_EDX)
        
        try:
            buf = uc.mem_read(ecx, edx)
            try:
             i=len(buf)
             while(i>0):
              b=buf[i-4:i]
              print(struct.pack(">L", (struct.unpack("<L",b)[0])).decode('ascii'), end="")
                
              i-=4
             print("")
            except:
              pass
        except UcError as e:
            print(">>> 0x%x: interrupt 0x%x, SYS_WRITE. buffer = 0x%x, size = %u, content = <unknown>\n" \
                        %(eip, intno, ecx, edx))


In [25]:
xor_keys=[]
def emulate(code):

    try:
        # Initialize emulator
        mu = Uc(UC_ARCH_X86, UC_MODE_32)

        # map 2MB memory for this emulation
        mu.mem_map(ADDRESS, 2 * 1024 * 1024)

        # write machine code to be emulated to memory
        mu.mem_write(ADDRESS, code)

        # initialize stack
        mu.reg_write(UC_X86_REG_ESP, ADDRESS + 0x100000)

        # tracing all instructions with customized callback
        mu.hook_add(UC_HOOK_CODE, hook_code)

        # handle interrupt ourself
        mu.hook_add(UC_HOOK_INTR, hook_intr)

        # set key candiate to ESI
        mu.reg_write(UC_X86_REG_ESI, 0)
        
        # emulate machine code in infinite time
        mu.emu_start(ADDRESS, ADDRESS + len(code))

        # now print out some registers
        print(">>> Emulation done")

    except UcError as e:
        print("ERROR: %s" % e)

In [26]:
emulate(assemble(source))

tracing xor dword ptr [esp + 4], esi: esp + 4: bdaf6ce0, esi: 0x0
esp + 4 after XOR: bdaf6ce0, bytearray(b'\xbd\xafl\xe0')

tracing xor dword ptr [esp + 4], esi: esp + 4: 567d8dde, esi: 0xe8e9e9e4
esp + 4 after XOR: b2946436, bytearray(b'\xb2\x94d6')

tracing xor dword ptr [esp + 4], esi: esp + 4: 624e8747, esi: 0x1716161c
esp + 4 after XOR: 7e589150, bytearray(b'~X\x91P')

tracing xor dword ptr [esp + 4], esi: esp + 4: 52449765, esi: 0x8a0b2b2c
esp + 4 after XOR: 7e6f9cef, bytearray(b'~o\x9c\xef')

tracing xor dword ptr [esp + 4], esi: esp + 4: 4b428d7e, esi: 0x8e9e99f
esp + 4 after XOR: d4ab6476, bytearray(b'\xd4\xabdv')

tracing xor dword ptr [esp + 4], esi: esp + 4: 52ec593e, esi: 0x99a88cce
esp + 4 after XOR: 9c60f1a7, bytearray(b'\x9c`\xf1\xa7')

tracing xor dword ptr [esp + 4], esi: esp + 4: a6ae38ab, esi: 0x78b5b1d5
esp + 4 after XOR: 731f8dd3, bytearray(b's\x1f\x8d\xd3')

tracing xor dword ptr [esp + 4], esi: esp + 4: ed264d50, esi: 0xab80a0a8
esp + 4 after XOR: 4586cdfb, byte

In [8]:
flag="flag{area_41_2018_stay_curious_and_keep_searching..}"

In [9]:
i=0
flag_tokens=[]
while(i<len(flag)):
    token=flag[i:i+4]
   
    flag_tokens.append(struct.unpack(">L",token.encode('ascii'))[0])
    
    i+=4

In [10]:
w=zip(flag_tokens,xor_keys)

In [11]:
for a in w:
    p=struct.pack(">L", a[0] ^ a[1]).hex()
    print(f"cipher text block to replace: 0x{p}")

cipher text block to replace: 0x666c6167
cipher text block to replace: 0x93889b81
cipher text block to replace: 0x7649222d
cipher text block to replace: 0xd5391b1d
cipher text block to replace: 0x30b69aeb
cipher text block to replace: 0xf8d1d3ad
cipher text block to replace: 0x0dc7d8ba
cipher text block to replace: 0xdef3ffc9
cipher text block to replace: 0x6af249d9
cipher text block to replace: 0x7483751c
cipher text block to replace: 0x2211a69f
cipher text block to replace: 0x50206eff
cipher text block to replace: 0xfe48b121


In [12]:
source_own = """

    jmp .forward
    .back:

    pop edi                  # edi has function ptr to .forward -> cipher function
    push byte 4              # push 4 to the stack
    pop eax                  # pop 4 off the stack to eax
    cdq                      # convert double to quad -> sign extension of value in eax to edx: The CDQ instruction copies the sign (bit 31) of the value in the EAX register into every bit position in the EDX register.
    mov ebx, edx             # edx = 0 -> ebx = 0
    inc ebx                  # ebx = 1
    mov ebp, 0x474f4f47      # initialize ebp -> 'GOOG'

    push 0x666c6167          # replaced with computed dw's from above
    call edi
    push 0x93889b81
    call edi
    push 0x7649222d
    call edi
    push 0xd5391b1d
    call edi
    push 0x30b69aeb
    call edi
    push 0xf8d1d3ad
    call edi
    push 0x0dc7d8ba
    call edi
    push 0xdef3ffc9
    call edi
    push 0x6af249d9
    call edi
    push 0x7483751c
    call edi
    push 0x2211a69f
    call edi
    push 0x50206eff
    call edi
    push 0xfe48b121
    call edi

    mov ecx, esp
    int 0x80
    mov eax, ebx
    int 0x80

    .forward: call .back
    xor [esp+4], esi
    add edx, 4
    xor ebp, esi
    ror ebp, 3
    sub ebp, edx
    xor esi, ebp
    ret
    """

In [13]:
emulate(assemble(source_own))

tracing xor dword ptr [esp + 4], esi: esp + 4: 67616c66, esi: 0x0
esp + 4 after XOR: 67616c66, bytearray(b'galf')

tracing xor dword ptr [esp + 4], esi: esp + 4: 819b8893, esi: 0xe8e9e9e4
esp + 4 after XOR: 6572617b, bytearray(b'era{')

tracing xor dword ptr [esp + 4], esi: esp + 4: 2d224976, esi: 0x1716161c
esp + 4 after XOR: 31345f61, bytearray(b'14_a')

tracing xor dword ptr [esp + 4], esi: esp + 4: 1d1b39d5, esi: 0x8a0b2b2c
esp + 4 after XOR: 3130325f, bytearray(b'102_')

tracing xor dword ptr [esp + 4], esi: esp + 4: eb9ab630, esi: 0x8e9e99f
esp + 4 after XOR: 74735f38, bytearray(b'ts_8')

tracing xor dword ptr [esp + 4], esi: esp + 4: add3d1f8, esi: 0x99a88cce
esp + 4 after XOR: 635f7961, bytearray(b'c_ya')

tracing xor dword ptr [esp + 4], esi: esp + 4: bad8c70d, esi: 0x78b5b1d5
esp + 4 after XOR: 6f697275, bytearray(b'oiru')

tracing xor dword ptr [esp + 4], esi: esp + 4: c9fff3de, esi: 0xab80a0a8
esp + 4 after XOR: 615f7375, bytearray(b'a_su')

tracing xor dword ptr [esp + 4],